<a href="https://colab.research.google.com/github/simplysumanth/Kaggle_Competitions/blob/main/Cassava_Disease_Classification_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get data

In [1]:
##Get Data from Kaggle
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"simplysumanth","key":"4c4d5aafd1e3c9c5586c80c0cbb778a3"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c cassava-disease

 98% 507M/515M [00:02<00:00, 219MB/s]
100% 515M/515M [00:02<00:00, 211MB/s]
 99% 1.04G/1.04G [00:04<00:00, 192MB/s]
100% 1.04G/1.04G [00:04<00:00, 226MB/s]
  0% 0.00/645k [00:00<?, ?B/s]
100% 645k/645k [00:00<00:00, 205MB/s]
  0% 0.00/83.8k [00:00<?, ?B/s]
100% 83.8k/83.8k [00:00<00:00, 86.9MB/s]
100% 776M/777M [00:21<00:00, 16.5MB/s]
100% 777M/777M [00:21<00:00, 37.5MB/s]


In [4]:
!unzip "train.zip" -d "/content/train"

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/train/cgm/train-cgm-519.jpg  
  inflating: /content/train/train/cgm/train-cgm-727.jpg  
  inflating: /content/train/train/cgm/train-cgm-733.jpg  
  inflating: /content/train/train/cgm/train-cgm-737.jpg  
  inflating: /content/train/train/cgm/train-cgm-723.jpg  
  inflating: /content/train/train/cgm/train-cgm-535.jpg  
  inflating: /content/train/train/cgm/train-cgm-253.jpg  
  inflating: /content/train/train/cgm/train-cgm-247.jpg  
  inflating: /content/train/train/cgm/train-cgm-521.jpg  
  inflating: /content/train/train/cgm/train-cgm-509.jpg  
  inflating: /content/train/train/cgm/train-cgm-290.jpg  
  inflating: /content/train/train/cgm/train-cgm-284.jpg  
  inflating: /content/train/train/cgm/train-cgm-441.jpg  
  inflating: /content/train/train/cgm/train-cgm-327.jpg  
  inflating: /content/train/train/cgm/train-cgm-333.jpg  
  inflating: /content/train/train/cgm/train-cgm-455.jpg  
  inflating: /content

In [5]:
!unzip "test.zip" -d "/content/test"

Archive:  test.zip
   creating: /content/test/test/
   creating: /content/test/test/0/
  inflating: /content/test/test/0/test-img-2397.jpg  
  inflating: /content/test/test/0/test-img-3089.jpg  
  inflating: /content/test/test/0/test-img-1846.jpg  
  inflating: /content/test/test/0/test-img-863.jpg  
  inflating: /content/test/test/0/test-img-877.jpg  
  inflating: /content/test/test/0/test-img-1852.jpg  
  inflating: /content/test/test/0/test-img-2383.jpg  
  inflating: /content/test/test/0/test-img-687.jpg  
  inflating: /content/test/test/0/test-img-693.jpg  
  inflating: /content/test/test/0/test-img-1885.jpg  
  inflating: /content/test/test/0/test-img-2354.jpg  
  inflating: /content/test/test/0/test-img-2432.jpg  
  inflating: /content/test/test/0/test-img-678.jpg  
  inflating: /content/test/test/0/test-img-2426.jpg  
  inflating: /content/test/test/0/test-img-3738.jpg  
  inflating: /content/test/test/0/test-img-2340.jpg  
  inflating: /content/test/test/0/test-img-1649.jpg  


In [6]:
import os
classes = []
for i in os.walk("/content/train/train"):
  classes = i[1]
  break

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms, datasets, models
from tqdm.notebook import tqdm as tqdm
import glob
import os

###Try to visualize the images

In [8]:
TRAIN_PATH = "/content/train/train"
TEST_PATH = "/content/test/test/0"

In [9]:
sub_paths = os.listdir(TRAIN_PATH)
sub_paths

['cbsd', 'cgm', 'cmd', 'healthy', 'cbb']

In [ ]:
os.listdir(TEST_PATH)

In [11]:
train_data_list, test_data_list = [], []
for c in sub_paths:
  train_data_list.extend(glob.glob(os.path.join(TRAIN_PATH+"/"+c,'*.jpg')))
print(len(train_data_list))

test_data_list = glob.glob(os.path.join(TEST_PATH,'*.jpg'))
print(len(test_data_list))

5656
3774


In [12]:
#Lets first check the amount of training images in each class
for c in sub_paths:
  print(f'The number of images in class {c}: {len(os.listdir(TRAIN_PATH+"/"+c))}')

The number of images in class cbsd: 1443
The number of images in class cgm: 773
The number of images in class cmd: 2658
The number of images in class healthy: 316
The number of images in class cbb: 466


**Inference**: Looks like there's some imbalance in the number of training data between each class\

Lets keep this is mind,and try to solve this, once our baseline model is created 

In [13]:
## We got the training images list

##this is how we can get the label
print(train_data_list[0].split('-')[-2])
print(test_data_list[0])

cbsd
/content/test/test/0/test-img-1060.jpg


In [14]:
### SPLIT into TRAIN,VALID,TEST
from sklearn.model_selection import train_test_split
train_data_list,valid_data_list = train_test_split(train_data_list,test_size = 0.2,random_state=42)

len(train_data_list), len(valid_data_list)

(4524, 1132)

In [15]:
##Lets look into an image
from PIL import Image
import cv2

print(cv2.imread(train_data_list[5]).shape)
Image.open(train_data_list[5])

for i in range(len(train_data_list)):
  if cv2.imread(train_data_list[i]).shape != (666, 500, 3):
    print(cv2.imread(train_data_list[i]).shape)
    break

(500, 625, 3)
(500, 514, 3)


**Inference**: The images are of different sizes, so we have to Crop/Resize them

In [16]:
### Lets define transforms to the images
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

valid_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [17]:
class CustomDataset(Dataset):
  def __init__(self,file_list,transform=None):
    self.file_list = file_list
    self.transform = transform
  
  def __len__(self):
    return len(self.file_list)
  
  def __getitem__(self,idx):
    class_list = ['cbsd', 'cgm', 'cmd', 'healthy', 'cbb']
    img_path = self.file_list[idx]
    img = Image.open(img_path)
    transformed_img = self.transform(img)

    label = self.file_list[idx].split('-')[-2]
    if label in class_list:
      label = class_list.index(label)
    else:
      label = self.file_list[idx].split('/')[-1]

    return {"image":transformed_img, "label":label}


In [18]:
train_data = CustomDataset(train_data_list,train_transform)
valid_data = CustomDataset(valid_data_list,valid_transform)
test_data = CustomDataset(test_data_list,test_transform)

In [ ]:
train_data[0].shape

In [29]:
print(train_data[0]["image"].shape, valid_data[0]["image"].shape, test_data[0]["image"].shape)
train_data[0]["label"], valid_data[0]["label"], test_data[0]["label"]

torch.Size([3, 224, 224]) torch.Size([3, 224, 224]) torch.Size([3, 224, 224])


(3, 2, 'test-img-1060.jpg')

In [23]:
#I think our data is ready!, Now we have to feed it to a dataloader
train_loader = DataLoader(train_data,batch_size=64,shuffle=True)
val_loader = DataLoader(valid_data,batch_size=64,shuffle=True)
test_loader = DataLoader(test_data,batch_size=64)

In [24]:
len(train_loader), len(val_loader), len(test_loader)

(71, 18, 59)

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model = models.resnet34(pretrained=True)
model.parameters

In [33]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
model.parameters

In [34]:
#Resnet
model.fc = nn.Sequential(
    nn.Linear(2048,512),
    nn.ReLU(),
    nn.Linear(512,128),
    nn.ReLU(),
    nn.Linear(128,5)
)

#Efficientnet
# model._fc = nn.Sequential(
#     nn.Linear(model._fc.in_features,512),
#     nn.ReLU(),
#     nn.Linear(512,128),
#     nn.ReLU(),
#     nn.Linear(128,5)
# )

In [ ]:
model = model.to(device)
model.train()

In [36]:
#SET Optimizer & Loss fn
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [57]:
epochs = 15

for epoch in range(epochs):
  epoch_loss, epoch_accuracy = 0,0
  tk= tqdm(train_loader,desc = "Train Iteration")

  for idx, batch in enumerate(tk):
    # print(data["image"])
    # print(data["label"])
    print(batch)
    print(label)
    data = data["image"].to(device)
    label = data["label"].to(device)
    
    output = model(data)
    loss = criterion(output,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = ((output.argmax(dim=1) == label).float().mean())
    epoch_accuracy += acc/len(train_loader)
    epoch_loss += loss/len(train_loader)

  print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))

  with torch.no_grad():
    epoch_val_accuracy, epoch_val_loss = 0,0
    vk = tqdm(val_loader, desc = "Valid Iteration")

    for idx, data in enumerate(vk):
      data = data["image"].to(device)
      label = data["label"].to(device)

      val_output = model(data)
      val_loss = criterion(val_output,label)

      acc = ((val_output.argmax(dim=1) == label).float().mean())
      epoch_val_accuracy += acc/len(val_loader)
      epoch_val_loss += val_loss/len(val_loader)

  print('Epoch : {}, val accuracy : {}, val loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))


{'image': tensor([[[[0.4588, 0.4314, 0.4078,  ..., 0.5961, 0.5647, 0.5373],
          [0.4549, 0.4275, 0.4039,  ..., 0.6118, 0.6000, 0.6196],
          [0.4549, 0.4235, 0.4000,  ..., 0.6314, 0.6196, 0.6471],
          ...,
          [0.5412, 0.4902, 0.5725,  ..., 0.5451, 0.5647, 0.5765],
          [0.5216, 0.5255, 0.6118,  ..., 0.5137, 0.5490, 0.5647],
          [0.5451, 0.5804, 0.6588,  ..., 0.4902, 0.5333, 0.5686]],

         [[0.4824, 0.4588, 0.4353,  ..., 0.6353, 0.6039, 0.5608],
          [0.4784, 0.4549, 0.4314,  ..., 0.6431, 0.6275, 0.6392],
          [0.4784, 0.4549, 0.4275,  ..., 0.6588, 0.6431, 0.6549],
          ...,
          [0.6039, 0.5490, 0.6196,  ..., 0.5490, 0.5647, 0.5725],
          [0.5882, 0.5882, 0.6667,  ..., 0.4980, 0.5255, 0.5412],
          [0.6157, 0.6471, 0.7216,  ..., 0.4627, 0.4980, 0.5294]],

         [[0.1333, 0.1255, 0.1255,  ..., 0.2353, 0.2431, 0.2471],
          [0.1294, 0.1216, 0.1255,  ..., 0.2392, 0.2667, 0.3255],
          [0.1333, 0.1216, 0.125

TypeError: ignored

In [32]:
torch.save(model.state_dict(),'efficinet_b07_weights.pth')

In [39]:
from tqdm.notebook import tqdm as tqdm

In [42]:
model.eval()
tk= tqdm(test_loader,desc = "Epoch")

for step, batch in enumerate(tk):
  print(batch["image"])
  break

TypeError: ignored

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

  #print(type(batch[0][0]))
  #img = batch[0][8]
  #plt.imshow(img.permute(1, 2, 0))
  #plt.show()  
  #print(batch[1][8])
  print(batch_idx)
  print(batch)
  break

In [54]:
model.eval()
with torch.no_grad():
  test_images = os.listdir(TEST_PATH)
  print(test_images[0])
  img = Image.open(TEST_PATH+"/"+test_images[0])
  trans = test_transform(img)
  data = trans.to(device)
  preds = model(data)
  preds_list = F.softmax(preds,dim=1)[:1].tolist()
  label = preds_list[0].index(max(preds_list[0]))
  print(label)
  plt.imshow(trans.permute(1, 2, 0))
  plt.show()  

test-img-1060.jpg


RuntimeError: ignored

In [44]:
leaves_probs = []
model.eval()
c = 0
with torch.no_grad():
  
    data = data.to(device)
    preds = model(data)
    preds_list = F.softmax(preds,dim=1)[:1].tolist()
    label = preds_list[0].index(max(preds_list[0]))
    if c == 10:
      break
    else: 
      print(label,fileid)
    
    #leaves_probs += list(zip(list(fileid),preds_list))


test-img-1060.jpg
1 ('test-img-1060.jpg', 'test-img-3619.jpg', 'test-img-1017.jpg', 'test-img-2297.jpg', 'test-img-1740.jpg', 'test-img-3384.jpg', 'test-img-2370.jpg', 'test-img-900.jpg', 'test-img-464.jpg', 'test-img-185.jpg', 'test-img-2609.jpg', 'test-img-183.jpg', 'test-img-578.jpg', 'test-img-6.jpg', 'test-img-3009.jpg', 'test-img-704.jpg', 'test-img-2525.jpg', 'test-img-1960.jpg', 'test-img-3555.jpg', 'test-img-1079.jpg', 'test-img-2244.jpg', 'test-img-1399.jpg', 'test-img-2852.jpg', 'test-img-5.jpg', 'test-img-1713.jpg', 'test-img-2674.jpg', 'test-img-401.jpg', 'test-img-2784.jpg', 'test-img-645.jpg', 'test-img-2247.jpg', 'test-img-3419.jpg', 'test-img-2069.jpg', 'test-img-1926.jpg', 'test-img-711.jpg', 'test-img-340.jpg', 'test-img-628.jpg', 'test-img-2624.jpg', 'test-img-3752.jpg', 'test-img-2647.jpg', 'test-img-3477.jpg', 'test-img-823.jpg', 'test-img-898.jpg', 'test-img-3048.jpg', 'test-img-2418.jpg', 'test-img-2637.jpg', 'test-img-200.jpg', 'test-img-3706.jpg', 'test-img-25